# Try out using pytorch

In [7]:
import pandas as pd
df = pd.read_csv("../data/processed_data.csv")
# The set of all metrics available.
all_met = ['clean_prompt_length','clean_response_a_length','clean_response_b_length','prompt_length','response_a_length','response_b_length','length_diff','prompt_sentiment','response_a_sentiment','response_b_sentiment','response_a_readability','response_b_readability','readability_diff']
len(all_met)
def get_lab(row):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1
    else:
        return 2

# Apply them to the dataframe
df['label'] = df.apply(get_lab, axis=1)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import itertools



In [9]:


class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

def torch_eval(feat):
    X = df[feat].to_numpy()
    y = df['label'].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)

    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    device = torch.device("cuda")
    model = MLPClassifier(input_dim=len(feat), hidden_dim=64, output_dim=3).to(device)

    # train
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    num_epochs = 3
    batch_size = 64

    for epoch in range(num_epochs):
        model.train()
        permutation = torch.randperm(X_train_tensor.size(0))

        for i in range(0, X_train_tensor.size(0), batch_size):
            indices = permutation[i:i+batch_size]
            batch_X = X_train_tensor[indices].to(device)
            batch_y = y_train_tensor[indices].to(device)

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor.to(device))
        predictions = torch.argmax(outputs, dim=1).cpu()

    report = classification_report(y_test, predictions, output_dict=True)
    print(report['accuracy'])
    res = float(report['accuracy'])
    return res, model, scaler

def all_sub(lis):
    yield from itertools.chain.from_iterable(itertools.combinations(lis, r) for r in range(len(lis)+1))

maxe = 0
bst_lis = []
for s in all_sub(all_met):
    if len(s) == 0:
        continue
    if "length_diff" not in s:
        continue
    if len(s) < 7:
        continue
    print("Now feat: ", s)
    ans,_,__ = torch_eval(list(s))
    if ans > maxe:
        maxe = ans
        bst_lis = s
    print(ans)
    print("Current max: ", maxe, bst_lis)


Now feat:  ('clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'response_a_length', 'response_b_length', 'length_diff')
Epoch 1/3, Loss: 1.0046
Epoch 2/3, Loss: 1.0749
Epoch 3/3, Loss: 1.0698
0.4484168406402227
0.4484168406402227
Current max:  0.4484168406402227 ('clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'response_a_length', 'response_b_length', 'length_diff')
Now feat:  ('clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'response_a_length', 'length_diff', 'prompt_sentiment')
Epoch 1/3, Loss: 1.1142
Epoch 2/3, Loss: 1.0093
Epoch 3/3, Loss: 1.0985
0.4437195546276966
0.4437195546276966
Current max:  0.4484168406402227 ('clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'response_a_length', 'response_b_length', 'length_diff')
Now feat:  ('clean_prompt_length', 'clean_response_a_length', 'clean_response_b_lengt

KeyboardInterrupt: 

Decent results we got:
```Current max:  0.4497216423103688 ('clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'length_diff', 'prompt_sentiment', 'readability_diff')```


In [10]:
_, model, scaler = torch_eval(['clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'length_diff', 'prompt_sentiment', 'readability_diff'])


Epoch 1/3, Loss: 1.0982
Epoch 2/3, Loss: 1.0509
Epoch 3/3, Loss: 1.1412
0.43771746694502434


In [11]:
import torch.nn.functional as F

testdf = pd.read_csv("../data/processed_test.csv")
ans = [[0,0,0],[0,0,0],[0,0,0]]

for i in range(200):
    _, model, scaler = torch_eval(['clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'length_diff', 'prompt_sentiment', 'readability_diff'])
    X_test = testdf[['clean_prompt_length', 'clean_response_a_length', 'clean_response_b_length', 'prompt_length', 'length_diff', 'prompt_sentiment', 'readability_diff']].values  # shape: [num_samples, 7]
    X_test_scaled = scaler.transform(X_test)
    device = torch.device("cuda")
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        logits = model(X_test_tensor)
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    print(preds)

Epoch 1/3, Loss: 1.1545
Epoch 2/3, Loss: 1.0366
Epoch 3/3, Loss: 1.0475
0.4375434933890049
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 1.0361
Epoch 2/3, Loss: 1.0834
Epoch 3/3, Loss: 1.0742
0.44041405706332637
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 1.0225
Epoch 2/3, Loss: 1.0313
Epoch 3/3, Loss: 1.0347
0.4425017397355602
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 1.0615
Epoch 2/3, Loss: 1.0093
Epoch 3/3, Loss: 1.0675
0.43797842727905356
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 1.1061
Epoch 2/3, Loss: 1.0873
Epoch 3/3, Loss: 0.9811
0.4396311760612387
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 0.9407
Epoch 2/3, Loss: 1.1156
Epoch 3/3, Loss: 1.1430
0.4352818371607516
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 1.1028
Epoch 2/3, Loss: 1.1239
Epoch 3/3, Loss: 1.1203
0.447633959638135
tensor([2, 0, 0], device='cuda:0')
Epoch 1/3, Loss: 1.2019
Epoch 2/3, Loss: 1.0773
Epoch 3/3, Loss: 1.1446
0.4462421711899791
tensor([2, 0, 0], device='

KeyboardInterrupt: 